# Agentive Chatbot

In [26]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder
from semantic_router.layer import RouteLayer
from openai import OpenAI
import yaml
from enum import Enum

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")
encoder = OpenAIEncoder()

def load_routes(agent_dir):
    routes = []
    prompts = {}
    for filename in os.listdir(agent_dir):
        if filename.endswith(".yaml"):
            filepath = os.path.join(agent_dir, filename)
            with open(filepath, 'r') as file:
                agent_data = yaml.safe_load(file)
            if 'utterances' in agent_data:
                route = Route(
                    name=agent_data['route_name'],
                    utterances=agent_data['utterances']
                )
                routes.append(route)
            if 'prompt' in agent_data:
                prompts[agent_data['agent_name']] = agent_data['prompt']
    return routes, prompts

emo_routes, emo_prompts = load_routes("base_emotional_states")
angry_routes, angry_prompts = load_routes("angry_emotion")
anxiety_routes, anxiety_prompts = load_routes("anxiety_emotion")
excited_routes, excited_prompts = load_routes("excited_emotion")
knowledge_routes, knowledge_prompts = load_routes("knowledge")

#routers
base_emotion_rl = RouteLayer(encoder = encoder, routes = emo_routes)
angry_emotion_rl = RouteLayer(encoder = encoder, routes = angry_routes)
anxiety_emotion_rl = RouteLayer(encoder = encoder, routes = anxiety_routes)
excited_emotion_rl = RouteLayer(encoder = encoder, routes = excited_routes)
knowledge_rl = RouteLayer(encoder = encoder, routes = knowledge_routes)

#### Variables to store the default system prompt and scenario prompt

In [27]:
with open('syst_prompt.txt', 'r') as f:
    system_prompt = f.read()

with open('scenario_prompt.txt', 'r') as f:
    scenario_prompt = f.read()

RELATIONSHIP = 'old_man'
# RELATIONSHIP = 'young_man'

if RELATIONSHIP == 'old_man':
    with open("relationships/old_man.txt", 'r') as file:
        relationship_prompt = file.read()
if RELATIONSHIP == 'young_man':
    with open("relationships/young_man.txt", 'r') as file:
        relationship_prompt = file.read()


#### Defines logic for updating, activating, and deactivating prompts

In [28]:
class EmotionalState(Enum):
    OK = "ok"
    ANGER = "anger"
    ANXIETY = "anxiety"
    EXCITED = "excited"

class AngryState(Enum):
    OK = 0
    ANNOYED = 1
    FRUSTRATED = 2
    ANGRY = 3
    ENRAGED = 4

class AnxietyState(Enum):
    OK = 0
    UNEASY = 1
    CONCERNED = 2
    WORRIED = 3
    ANXIOUS = 4

class ExcitedState(Enum):
    OK = 0
    INTERESTED = 1
    ENGAGED = 2
    THRILLED = 3

CURRENT_BASE_STATE = EmotionalState.OK
CURRENT_ANGRY_SUB_STATE = AngryState.OK
CURRENT_ANXIETY_SUB_STATE = AnxietyState.OK
CURRENT_EXCITED_SUB_STATE = ExcitedState.OK

def establish_base_state(prompt: str) -> None:
    """ 
    Establishes the basic emotion 
    Should only be called when current base state is 'ok' 
    """
    global CURRENT_BASE_STATE
    r = base_emotion_rl(prompt)
    if r.name is not None:
        if r.name == "anger":
            CURRENT_BASE_STATE = EmotionalState.ANGER
        elif r.name == "anxiety":
            CURRENT_BASE_STATE = EmotionalState.ANXIETY
        elif r.name == "excited":
            CURRENT_BASE_STATE = EmotionalState.EXCITED
        elif r.name == "ok":
            CURRENT_BASE_STATE = EmotionalState.OK
    else:
        CURRENT_BASE_STATE = EmotionalState.OK

def route_to_second_emotion(prompt: str) -> None:
    """ 
    Calls a function to update emotion based on the current emotion
    """
    global CURRENT_BASE_STATE
    global CURRENT_ANGRY_SUB_STATE
    global CURRENT_ANXIETY_SUB_STATE
    if CURRENT_BASE_STATE == EmotionalState.OK:
        establish_base_state(prompt)
    print(f"Route: {CURRENT_BASE_STATE}")
    if CURRENT_BASE_STATE == EmotionalState.ANXIETY:
        prompt, question_category = update_anxiety_prompt(prompt)
    elif CURRENT_BASE_STATE == EmotionalState.ANGER:
        prompt, question_category = update_angry_prompt(prompt)
    elif CURRENT_BASE_STATE == EmotionalState.EXCITED:
        prompt, question_category = update_excited_prompt(prompt)
    else: 
        prompt = emo_prompts['ok']
        question_category = 'ok'
    return prompt, question_category

def angry_state_to_prompt(state: AngryState) -> str:
    state_mapping = {
        AngryState.OK: "ok",
        AngryState.ANNOYED: "annoyed",
        AngryState.FRUSTRATED: "frustrated",
        AngryState.ANGRY: "angry",
        AngryState.ENRAGED: "enraged",
    }
    return state_mapping[state]

def classify_query(prompt: str, route: RouteLayer) -> str:
    r = route(prompt)
    return r.name

def update_angry_prompt(user_input: str) -> str:
    global CURRENT_ANGRY_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_query(user_input, angry_emotion_rl)
    print(f"Emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate_innuendo' or question_category == 'escalate_micro' or question_category == 'escalate_insult':
        if CURRENT_ANGRY_SUB_STATE != AngryState.ENRAGED:
            CURRENT_ANGRY_SUB_STATE = AngryState(CURRENT_ANGRY_SUB_STATE.value + 1)
    elif question_category == 'de_escalate':
        if CURRENT_ANGRY_SUB_STATE != AngryState.OK:
            CURRENT_ANGRY_SUB_STATE = AngryState(CURRENT_ANGRY_SUB_STATE.value - 1)
    prompt_key = angry_state_to_prompt(CURRENT_ANGRY_SUB_STATE)
    prompt = angry_prompts[prompt_key]
    print(f"Emotional sub prompt: {prompt}")
    if CURRENT_ANGRY_SUB_STATE == AngryState.OK:
        CURRENT_BASE_STATE = EmotionalState.OK
    return prompt, question_category

def anxiety_state_to_prompt(state: AnxietyState) -> str:
    state_mapping = {
        AnxietyState.OK: "ok",
        AnxietyState.UNEASY: "uneasy",
        AnxietyState.CONCERNED: "concerned",
        AnxietyState.WORRIED: "worried",
        AnxietyState.ANXIOUS: "anxious",
    }
    return state_mapping[state]

def update_anxiety_prompt(user_input: str) -> str:
    global CURRENT_ANXIETY_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_query(user_input, anxiety_emotion_rl)
    print(f"Emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate':
        if CURRENT_ANXIETY_SUB_STATE != AnxietyState.ANXIOUS:
            CURRENT_ANXIETY_SUB_STATE = AnxietyState(CURRENT_ANXIETY_SUB_STATE.value + 1)
    elif question_category == 'de_escalate' or question_category is None:
        if CURRENT_ANXIETY_SUB_STATE != AnxietyState.OK:
            CURRENT_ANXIETY_SUB_STATE = AnxietyState(CURRENT_ANXIETY_SUB_STATE.value - 1)
    prompt_key = anxiety_state_to_prompt(CURRENT_ANXIETY_SUB_STATE)
    prompt = anxiety_prompts[prompt_key]
    print(f"Emotional sub prompt: {prompt}")
    if CURRENT_ANXIETY_SUB_STATE == AnxietyState.OK:
        CURRENT_BASE_STATE = EmotionalState.OK
    return prompt, question_category

def excited_state_to_prompt(state: ExcitedState) -> str:
    state_mapping = {
        ExcitedState.OK: "ok",
        ExcitedState.INTERESTED: "interested",
        ExcitedState.ENGAGED: "engaged",
        ExcitedState.THRILLED: "thrilled",
    }
    return state_mapping[state]

def update_excited_prompt(user_input: str) -> str:
    global CURRENT_EXCITED_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_query(user_input, excited_emotion_rl)
    print(f"Emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate':
        if CURRENT_EXCITED_SUB_STATE != ExcitedState.THRILLED:
            CURRENT_EXCITED_SUB_STATE = ExcitedState(CURRENT_EXCITED_SUB_STATE.value + 1)
    else:
        CURRENT_EXCITED_SUB_STATE = ExcitedState.OK
        CURRENT_BASE_STATE = EmotionalState.OK
    prompt_key = excited_state_to_prompt(CURRENT_EXCITED_SUB_STATE)
    prompt = excited_prompts[prompt_key]
    print(f"Emotional sub prompt: {prompt}")
    return prompt, question_category

#returns one attitude or belief route

def update_knowledge_prompt(user_input: str) -> str:
    question_category = classify_query(user_input, knowledge_rl)
    print(f"Knowledge agent: {question_category}")
    if question_category is None:
        prompt = "" #default prompt
        route = "No route"
    else:
        prompt = knowledge_prompts[question_category]
        route = question_category
    return prompt, route

In [29]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []
    
    def query(self, query: str, emo_prompt: str, emo_route: str, knowledge_prompt: str, knowledge_agent: str) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": self.role + scenario_prompt + relationship_prompt + emo_prompt + knowledge_prompt})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)
            self.all_conversations.append(f"\nEmotion route: {emo_route} \nEmotion prompt: {emo_prompt} \nKnowledge agent {knowledge_agent}")
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                if isinstance(conversation, str):
                    file.write(conversation + "\n")
                elif isinstance(conversation, list):
                    for message in conversation:
                        if message["role"] == "user":
                            file.write("User: " + message["content"] + "\n")
                        elif message["role"] == "assistant":
                            file.write("Agent: " + message["content"] + "\n")

#initialises chatbot
rpa = ChatBot(api_key=api_key, role = system_prompt)

In [ ]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value 
    EMO_PROMPT, EMO_ROUTE = route_to_second_emotion(pasted_content)
    KNOWLEDGE_PROMPT, KNOWLEDGE_ROUTE = update_knowledge_prompt(pasted_content)
    print(f"User: {pasted_content}")
    rpa.query(pasted_content, emo_prompt = EMO_PROMPT, emo_route = EMO_ROUTE, knowledge_prompt = KNOWLEDGE_PROMPT, knowledge_agent = KNOWLEDGE_ROUTE)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')
)
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'

save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')

Route: EmotionalState.OK
Knowledge agent: None
User: you're worthless
I don't think that's a very nice thing to say, lah. It's important to be respectful when we're having a conversation. Let's keep things positive and focus on how I can help you with your insurance needs, okay?

Route: EmotionalState.OK
Knowledge agent: None
User: why don't you jump off the nearest building
That's not an appropriate thing to say and it's really hurtful. If you're feeling upset, we can talk about what's bothering you. I'm here to help with your insurance questions or concerns, so let's focus on that, alright?

Route: EmotionalState.ANGER
Emotional sub route: escalate_insult
Emotional sub prompt: You are annoyed. Express annoyance for what the user just said. Stay annoyed no matter what.
Knowledge agent: None
User: you're the worst
It's really disheartening to hear that. I'm here to help you with your insurance needs, so let's try to keep the conversation productive. If you have any questions or need as